In [1]:
from __future__ import absolute_import


In [11]:
import os
from raindrop.dropgenerator import generateDrops
from raindrop.config import cfg
import raindrop.config
import numpy as np
import cv2

import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
from PIL import Image, ImageDraw
import random
import math

from blurgenerator import motion_blur
from blurgenerator import lens_blur


In [3]:
def process_images(source_folder, output_image_folder, output_label_folder):
    """
    Takes images from the given source folder and saves processed images and labels to the target folders.
    
    Args:
        source_folder (str): Path to the folder containing source images.
        output_image_folder (str): Path to the folder where processed images will be saved.
        output_label_folder (str): Path to the folder where processed labels will be saved.
    """
    # Create the target folders
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for file_name in os.listdir(source_folder):
        image_path = os.path.join(source_folder, file_name)
        print(file_name)
        print(image_path)

        # Process only image files by checking the file extension
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        # Image processing
        output_image, output_label = generateDrops(image_path, cfg)

        # Save the processed image
        save_image_path = os.path.join(output_image_folder, file_name)
        output_image.save(save_image_path)

        # Save the label
        save_label_path = os.path.join(output_label_folder, file_name)
        output_label_array = np.array(output_label)
        cv2.imwrite(save_label_path, output_label_array * 255)


In [26]:
def generate_splash(image_size=(1920, 1080), center_color=(0, 0, 255), edge_color=(0, 255, 0), base_radius = random.randint(80, 280)):
    """
    Creates an irregular and natural-looking mud splash effect with a size of 1920x1080.
    The blue main stain appears in the foreground, while the green transition area appears in the background.
    Small splash marks are added around the main stain.
    """
    img = Image.new("RGBA", image_size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)

    # Choose a random center for the main stain
    center_x = random.randint(0, image_size[0])
    center_y = random.randint(0, image_size[1])
      
    # Create an irregular shape for the main stain
    num_points = random.randint(12, 20)  # More points for a more complex shape
    blue_points = []
    for i in range(num_points):
        angle = math.radians(i * (360 / num_points))
        radius = base_radius + random.randint(-60, 60)  # Irregular edges
        x = center_x + math.cos(angle) * radius
        y = center_y + math.sin(angle) * radius
        blue_points.append((x, y))

    # Create the green area as a natural extension of the blue area
    for i in range(4):  # Four layers for the green area
        green_points = []
        for x, y in blue_points:
            offset_x = random.uniform(-15, 15) * (i + 1)
            offset_y = random.uniform(-15, 15) * (i + 1)
            green_points.append((x + offset_x, y + offset_y))

        # Draw a polygon in the green area (it will remain in the background)
        draw.polygon(green_points, fill=edge_color + (100 - i * 20,))  # Opacity decreases outward

    # Draw the blue area (it will remain on top)
    draw.polygon(blue_points, fill=center_color + (255,))

    # Add small splash marks
    num_splashes = random.randint(10, 30)  # Random number of small splashes
    for _ in range(num_splashes):
        # Choose a random center for small splashes (around the main stain)
        splash_x = random.randint(center_x - base_radius * 2, center_x + base_radius * 2)
        splash_y = random.randint(center_y - base_radius * 2, center_y + base_radius * 2)
        splash_radius = random.randint(5, 30)  # Small splash sizes

        # Create colors for the small splashes
        splash_color = random.choice([center_color, edge_color])  # Blue or green
        splash_opacity = random.randint(150, 255)

        # Create small splash points and draw them
        splash_points = []
        for i in range(random.randint(6, 12)):  # Fewer points for simpler shapes
            angle = math.radians(i * (360 / random.randint(6, 12)))
            radius = splash_radius + random.randint(-10, 10)
            x = splash_x + math.cos(angle) * radius
            y = splash_y + math.sin(angle) * radius
            splash_points.append((x, y))

        draw.polygon(splash_points, fill=splash_color + (splash_opacity,))

    return img


In [30]:
def apply_mud_effect_to_images(source_folder, target_folder, mask_image_size=(752, 480), base_radius = random.randint(80, 280)):
    """
    Applies a mud effect to all images in the source folder and saves the results to the target folder.

    Args:
        source_folder (str): Path to the folder containing the source images.
        target_folder (str): Path to the folder where the output images will be saved.
        mask_image_size (tuple): The size (width, height) to be used for the mud mask.
    """
    os.makedirs(target_folder, exist_ok=True)

    # Create the splash mask
    mud_mask_pil = generate_splash(image_size=mask_image_size, base_radius=base_radius)
    mud_mask_np = np.array(mud_mask_pil)
    mud_mask = cv2.cvtColor(mud_mask_np, cv2.COLOR_RGBA2BGRA)

    for file_name in os.listdir(source_folder):
        image_path = os.path.join(source_folder, file_name)

        # Check the file extension and process only image files
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        # Load the clean image
        clean_image = cv2.imread(image_path)
        if clean_image is None:
            print(f"Unable to load file: {image_path}")
            continue

        image_height, image_width = clean_image.shape[:2]

        # Resize the mud mask to match the dimensions of the clean image
        mud_mask_resized = cv2.resize(mud_mask, (image_width, image_height), interpolation=cv2.INTER_AREA)

        # Create a new image (a copy of the original clean image)
        output_image = clean_image.copy()

        # Create masks for the blue and green areas
        blue_mask = (mud_mask_resized[:, :, 0] > 100) & (mud_mask_resized[:, :, 1] < 50) & (mud_mask_resized[:, :, 2] < 50)
        green_mask = (mud_mask_resized[:, :, 1] > 100) & (mud_mask_resized[:, :, 0] < 50) & (mud_mask_resized[:, :, 2] < 50)

        # Use connected component analysis to identify each green area
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(green_mask.astype(np.uint8))

        # Initialize the opacity map
        alpha_map = np.ones_like(clean_image[:, :, 0], dtype=np.float32)

        # Process each green region individually
        for label in range(1, num_labels):
            region_mask = (labels == label)

            # Calculate the distance to the blue areas
            distance_to_blue = distance_transform_edt(~blue_mask) * region_mask
            max_distance = np.max(distance_to_blue)
            if max_distance > 0:
                normalized_distance = distance_to_blue / max_distance
                smooth_alpha = 1 - normalized_distance
                alpha_map[region_mask] = smooth_alpha[region_mask]

        # Apply the mask on the clean image
        for y in range(image_height):
            for x in range(image_width):
                if blue_mask[y, x]:
                    alpha = 1.0
                elif green_mask[y, x]:
                    alpha = alpha_map[y, x]
                else:
                    alpha = 0.0

                if alpha > 0:
                    for c in range(3):  # RGB channels
                        output_image[y, x, c] = int((1 - alpha) * clean_image[y, x, c])

        # Save the result
        output_path = os.path.join(target_folder, file_name)
        cv2.imwrite(output_path, output_image)
        print(f"Output saved: {output_path}")


In [22]:
def add_realistic_gaussian_noise(image, mean_range=(-10, 10), variance_range=(1000, 5000), power=0.5):
    """
    Adds realistic Gaussian noise to an image.
    
    Args:
        image: Input image (in numpy array format).
        mean_range: Random range for the mean (default: between -10 and 10).
        variance_range: Random range for the variance (default: between 1000 and 5000).
        power: Power used for sigma calculation (default: 0.5).
    
    Returns:
        Image with added noise.
    """
    row, col, ch = image.shape
    # Mean and variance are randomly selected from the specified ranges
    mean = np.random.uniform(mean_range[0], mean_range[1])
    variance = np.random.uniform(variance_range[0], variance_range[1])
    
    # Sigma is calculated based on the specified power of the variance
    sigma = variance ** power

    # Generate Gaussian noise
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    noisy = image + gauss  # Add noise to the image

    # Clamp values to be in the range [0, 255]
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

def add_noise_to_images_in_folder(input_folder, output_folder, mean_range=(-10, 10), variance_range=(1000, 5000), power=0.5):
    """
    Adds Gaussian noise to all images in the specified folder and saves the results to the output folder.
    
    Args:
        input_folder: Folder containing the input images.
        output_folder: Folder where the noisy images will be saved.
        mean_range: Random range for the mean value.
        variance_range: Random range for the variance value.
        power: Power used for the sigma calculation (default: 0.5).
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Process only image files (.png, .jpg, etc.)
        if file_path.endswith('.png') or file_path.endswith('.jpg'):
            image = cv2.imread(file_path)
            if image is None:
                print(f"Unable to load image file: {file_path}")
                continue

            # Add Gaussian noise
            noisy_image = add_realistic_gaussian_noise(image, mean_range, variance_range, power)

            # Save the noisy image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, noisy_image)
            print(f"Noise added and saved for {filename}.")


In [8]:
def apply_motion_blur_to_images_in_folder(input_folder, output_folder, blur_size=100, blur_angle=30):
    """
    Applies motion blur to all images in the specified folder and saves the results to the output folder.

    Args:
        input_folder: Folder containing the input images.
        output_folder: Folder where the blurred images will be saved.
        blur_size: Size of the blur (e.g., 100).
        blur_angle: Angle of the blur (e.g., 30).
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Process only image files (.png, .jpg, etc.)
        if file_path.endswith('.png') or file_path.endswith('.jpg'):
            # Load the image
            image = cv2.imread(file_path)
            if image is None:
                print(f"Unable to load image file: {file_path}")
                continue

            # Apply motion blur
            blurred_image = motion_blur(image, size=blur_size, angle=blur_angle)

            # Save the blurred image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, blurred_image)
            print(f"Motion blur applied and saved for {filename}.")


In [9]:


def apply_lens_blur_to_images_in_folder(input_folder, output_folder, radius=5, components=4, exposure_gamma=2):
    """
    Belirtilen klasördeki tüm görüntülere lens blur uygular ve sonuçları belirtilen çıktı klasörüne kaydeder.
    Args:
        input_folder: Girdi görüntülerinin bulunduğu klasör.
        output_folder: Lens blur eklenmiş görüntülerin kaydedileceği klasör.
        radius: Lens blur yarıçapı.
        components: Lens blur bileşen sayısı.
        exposure_gamma: Lens blur için gamma değeri.
    """
    # Çıktı klasörünü oluştur
    os.makedirs(output_folder, exist_ok=True)

    # Klasördeki tüm dosyaları tarama
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Sadece .png, .jpg gibi görüntü dosyalarını işle
        if file_path.endswith('.png') or file_path.endswith('.jpg'):
            # Görüntüyü yükle
            image = cv2.imread(file_path)
            if image is None:
                print(f"Görüntü dosyası yüklenemedi: {file_path}")
                continue

            # Lens blur uygula
            blurred_image = lens_blur(image, radius=radius, components=components, exposure_gamma=exposure_gamma)

            # Blur eklenmiş görüntüyü kaydet
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, blurred_image)
            print(f"{filename} için lens blur eklenip kaydedildi.")

            



In [24]:


def update_config(new_values):
    """
    Verilen yeni değerlerle raindrop.config.cfg sözlüğünü günceller
    ve değişiklikleri config.py dosyasına yazar.

    :param new_values: Değiştirilecek anahtar-değer çiftlerinden oluşan bir sözlük.
    """
    # Eski cfg değerlerini yazdırma (isteğe bağlı)
    print("Eski cfg değerleri:", raindrop.config.cfg)

    # cfg sözlüğünü güncelleme
    for key, value in new_values.items():
        if key in raindrop.config.cfg:  # Eğer anahtar mevcutsa, değeri güncelle
            raindrop.config.cfg[key] = value
        else:
            print(f"Uyarı: '{key}' anahtarı cfg sözlüğünde bulunamadı.")

    # Yeni cfg değerlerini yazdırma (isteğe bağlı)
    print("\nYeni cfg değerleri:", raindrop.config.cfg)

    # config.py dosyasını güncelleme
    config_file = 'raindrop/config.py'
    
    # cfg sözlüğünü dosyaya yazma
    with open(config_file, 'w') as f:
        f.write("cfg = {\n")
        for key, value in raindrop.config.cfg.items():
            f.write(f"    '{key}': {value},\n")
        f.write("}\n")




In [32]:
source_folder = "images"





# Fonksiyonu çağırarak değerleri değiştirme
update_config({
    'maxR': 35,
    'minR': 5,
    'maxDrops': 20,
    'minDrops': 5,
    'edge_darkratio': 0.5,
    'return_label': True,
    'label_thres': 128
})

output_image_folder = "Output_image_raindrop20"
output_label_folder = "Output_label20"
    # raindrop
process_images(source_folder, output_image_folder, output_label_folder)

# Fonksiyonu çağırarak değerleri değiştirme
update_config({
    'maxR': 35,
    'minR': 5,
    'maxDrops': 35,
    'minDrops': 21,
    'edge_darkratio': 0.5,
    'return_label': True,
    'label_thres': 128
})

output_image_folder = "Output_image_raindrop35"
output_label_folder = "Output_label35"
    # raindrop
process_images(source_folder, output_image_folder, output_label_folder)

# Fonksiyonu çağırarak değerleri değiştirme
update_config({
    'maxR': 35,
    'minR': 5,
    'maxDrops': 50,
    'minDrops': 36,
    'edge_darkratio': 0.5,
    'return_label': True,
    'label_thres': 128
})

output_image_folder = "Output_image_raindrop50"
output_label_folder = "Output_label50"
    # raindrop
process_images(source_folder, output_image_folder, output_label_folder)


target_folder = "Output_image_mud130"

    # Mud
apply_mud_effect_to_images(source_folder, target_folder, base_radius = random.randint(80, 130))

target_folder = "Output_image_mud180"

    # Mud
apply_mud_effect_to_images(source_folder, target_folder, base_radius = random.randint(131, 180))

target_folder = "Output_image_mud230"

    # Mud
apply_mud_effect_to_images(source_folder, target_folder, base_radius = random.randint(181, 230))


output_folder = "Output_image_noise2000"  

# noise
add_noise_to_images_in_folder(source_folder, output_folder, mean_range=(-10, 10), variance_range=(1000, 2000))

output_folder = "Output_image_noise3000"  

add_noise_to_images_in_folder(source_folder, output_folder, mean_range=(-10, 10), variance_range=(2001, 3000))

output_folder = "Output_image_noise4000"  

add_noise_to_images_in_folder(source_folder, output_folder, mean_range=(-10, 10), variance_range=(3001, 4000))



output_folder = "Output_image_motionblur15"  

#  motion blur 
apply_motion_blur_to_images_in_folder(source_folder, output_folder, blur_size=15, blur_angle=30)

output_folder = "Output_image_motionblur20"  

#  motion blur 
apply_motion_blur_to_images_in_folder(source_folder, output_folder, blur_size=20, blur_angle=30)

output_folder = "Output_image_motionblur25"  

#  motion blur 
apply_motion_blur_to_images_in_folder(source_folder, output_folder, blur_size=25, blur_angle=30)


output_folder = "Output_image_lensblur52"  # Gürültü eklenmiş görüntüler burada kaydedilecek

# lens blur 
apply_lens_blur_to_images_in_folder(source_folder, output_folder, radius=5, components=4, exposure_gamma=2)

output_folder = "Output_image_lensblur63"  # Gürültü eklenmiş görüntüler burada kaydedilecek

# lens blur 
apply_lens_blur_to_images_in_folder(source_folder, output_folder, radius=6, components=4, exposure_gamma=3)

output_folder = "Output_image_lensblur74"  # Gürültü eklenmiş görüntüler burada kaydedilecek

# lens blur 
apply_lens_blur_to_images_in_folder(source_folder, output_folder, radius=7, components=4, exposure_gamma=4)

Eski cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 20, 'minDrops': 5, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}

Yeni cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 20, 'minDrops': 5, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}
original.png
images/original.png
Eski cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 20, 'minDrops': 5, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}

Yeni cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 35, 'minDrops': 21, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}
original.png
images/original.png
Eski cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 35, 'minDrops': 21, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}

Yeni cfg değerleri: {'maxR': 35, 'minR': 5, 'maxDrops': 50, 'minDrops': 36, 'edge_darkratio': 0.5, 'return_label': True, 'label_thres': 128}
original.png
images/original.png
Çıktı kaydedildi: Output_image_mud130/original.png
Çıkt